# Problem Set 2, due April 20 at 11:59am (i.e., noon).

## Introduction to the assignment

For this assignment, you will be using data from the [Progresa program](http://en.wikipedia.org/wiki/Oportunidades), a government social assistance program in Mexico. This program, as well as the details of its impact, are described in the paper "[School subsidies for the poor: evaluating the Mexican Progresa poverty program](http://www.sciencedirect.com/science/article/pii/S0304387803001858)", by Paul Shultz (available on Canvas). Please familiarize yourself with the PROGRESA program before beginning this problem set, so you have a rough sense of where the data come from and how they were generated. If you just proceed into the problem set without understanding Progresa or the data, it will be very difficult!

The goal of this problem set is to implement some of the basic econometric techniques that you are learning in class to measure the impact of Progresa on secondary school enrollment rates. The timeline of the program was:

 * Baseline survey conducted in 1997
 * Intervention begins in 1998, "Wave 1" of data collected in 1998
 * "Wave 2 of data" collected in 1999
 * Evaluation ends in 2000, at which point the control villages were treated. 
 
When you are ready, download the progresa_sample.csv data from Canvas. The data are actual data collected to evaluate the impact of the Progresa program.  In this file, each row corresponds to an observation taken for a given child for a given year. There are two years of data (1997 and 1998), and just under 40,000 children who are surveyed in each year. For each child-year observation, the following variables are collected:

| Variable name | Description|
|------|------|
|year	  |year in which data is collected
|sex	  |male = 1|
|indig	  |indigenous = 1|
|dist_sec |nearest distance to a secondary school|
|sc	      |enrolled in school in year of survey|
|grc      |grade enrolled|
|fam_n    |family size|
|min_dist |	min distance to an urban center|
|dist_cap |	min distance to the capital|
|poor     |	poor = 1|
|progresa |treatment =1|
|hohedu	  |years of schooling of head of household|
|hohwag	  |monthly wages of head of household|
|welfare_index|	welfare index used to classify poor|
|hohsex	|gender of head of household (male=1)|
|hohage	|age of head of household|
|age	|years old|
|folnum	|individual id|
|village|	village id|
|sc97	|schooling in 1997|

---

## Part 1: Descriptive analysis

### 1.1	Summary Statistics

Present summary statistics (mean and standard deviation) for all of the demographic variables in the dataset (i.e., everything except year, folnum, village). Present these in a single table alphabetized by variable name. Do NOT simply expect the grader to scroll through your output!

In [5]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
prog_df = pd.read_csv('data/progresa_sample.csv')

In [6]:
summ_df = pd.DataFrame(columns = ['Mean', 'Standard Deviation'])
summ_df['Mean'] = prog_df.mean()
summ_df['Standard Deviation'] = prog_df.std()
remove = ['year', 'folnum', 'village']
summ_df = summ_df.drop(remove)

In [7]:
summ_df = summ_df.sort_index()
summ_df

,Mean,Standard Deviation
age,11.366460,3.167744
dist_cap,147.674452,76.063134
dist_sec,2.418910,2.234109
fam_n,7.215715,2.352900
grc,3.963537,2.499063
grc97,3.705372,2.572387
hohage,44.436717,11.620372
hohedu,2.768104,2.656106
hohsex,0.925185,0.263095
hohwag,586.985312,788.133664


In [37]:
prog_df.sex[2]

1.0

In [15]:
#prog_df.progresa[0:100]
myset = set(prog_df.progresa)
print(myset)
print(len(prog_df[prog_df.progresa == 'basal']))
print(len(prog_df[prog_df.progresa != 'basal']))

set(['basal', '0'])
47560
29690


In [26]:
poor_df = prog_df[prog_df['poor'] == 'pobre']


treatment_df = poor_df[poor_df['progresa'] == 'basal']
control_df = poor_df[poor_df['progresa'] != 'basal']



print(treatment_df.head(5))
print(control_df.head(5))
print(stats.ttest_ind(treatment_df.min_dist, control_df.min_dist, equal_var=False))

    year  sex  indig  dist_sec  sc  grc  fam_n    min_dist    dist_cap   poor  \
6     97    0      0     3.154   0    6      6  127.114780  154.196003  pobre   
7     98    0      0     3.154   0    6      6  127.114780  154.196003  pobre   
8     97    1      0     3.373   1    2      5   85.300272  105.878669  pobre   
9     98    1      0     3.373   1    2      5   85.300272  105.878669  pobre   
10    97    0      0     3.373   1    2      5   85.300272  105.878669  pobre   

    ...  hohedu  hohwag  welfare_index  hohsex  hohage  age  village  folnum  \
6   ...       4       0      684.00000       1      85   14      271       4   
7   ...       4       0      684.00000       1      85   15      271       4   
8   ...       6     875      742.14001       1      26    9      263       5   
9   ...       6     875      742.14001       1      26   10      263       5   
10  ...       6     875      742.14001       1      26    7      263       6   

    grc97  sc97  
6       6     

### 1.2 Differences at baseline?

Are the baseline (1997) demographic characteristics **for the poor**  different in treatment and control villages? Hint: Use a T-Test to determine whether there is a statistically significant difference in the average values of each of the variables in the dataset. Focus only on the data from 1997 for individuals who are poor (i.e., poor=='pobre').

Present your results in a single table with the following columns and 14 (or so) rows:

| Variable name | Average value (Treatment villages) | Average value (Control villages) | Difference (Treat - Control) | p-value |
|------|------|------|------|------|
|Male|?|?|?|?|


In [38]:
#subset df into poor
poor_df = prog_df[prog_df['poor'] == 'pobre']

#separating treatment and control group
treatment_df = poor_df[poor_df['progresa'] == 'basal']
control_df = poor_df[poor_df['progresa'] != 'basal']

#bad columns
remove = ['year', 'folnum', 'village', 'poor', 'progresa']

#
baseline_df = pd.DataFrame(columns = ['Variable name', 'Average value (Treatment villages)', 'Average value (Control villages)', 'Difference (Treat - Control)', 'p-value'])
for item in prog_df.columns:
    if item not in remove:
        baseline_df.loc[item] = [item, treatment_df[item].mean(), control_df[item].mean(), 
                                 treatment_df[item].mean() - control_df[item].mean(), 
                                 stats.ttest_ind(treatment_df[item], control_df[item], equal_var=False)[1]]
                                        
print(baseline_df)

               Variable name  Average value (Treatment villages)  \
sex                      sex                            0.519317   
indig                  indig                            0.325986   
dist_sec            dist_sec                            2.453122   
sc                        sc                            0.833891   
grc                      grc                            3.810007   
fam_n                  fam_n                            7.281327   
min_dist            min_dist                          107.152915   
dist_cap            dist_cap                          150.829074   
hohedu                hohedu                            2.663139   
hohwag                hohwag                          544.339544   
welfare_index  welfare_index                          655.428377   
hohsex                hohsex                            0.924656   
hohage                hohage                           43.648828   
age                      age                    

### 1.3 Interpretation

* A: Are there statistically significant differences between treatment and control villages as baseline? 
* B: Why does it matter if there are differences at baseline?
* C: What does this imply about how to measure the impact of the treatment?

*Discuss your results here*

### 1.4 Graphical exploration, part 1

For each level of household head education, compute the average enrollment rate in 1997. Create a scatterplot that shows this relationship. What do you notice?

In [ ]:
# Your code here

*Discuss your results here*

### 1.5 Graphical exploration, part 2

Create a histogram of village enrollment rates **among poor households in treated villages**, before and after treatment. Specifically, for each village, calculate the average rate of enrollment of poor households in treated villages in 1997, then compute the average rate of enrollment of poor households in treated villages in 1998. Create two separate histograms showing the distribution of these average enrollments rates, one histogram for 1997 and one histogram for 1998. On each histogram, draw a vertical line that intersects the x-axis at the average value (across all households). Does there appear to be a difference? Is this difference statistically significant?

In [ ]:
# Your code here

*Discuss your results here*

## Part 2: Measuring Impact

Our goal is to estimate the causal impact of the PROGRESA program on the social and economic outcomes of individuals in Mexico. We will focus on the impact of the program on the poor (those with poor=='pobre'), since only the poor were eligible to receive the PROGRESA assistance.

### 2.1 Simple differences: T-test

Begin by estimating the impact of Progresa using "simple differences." Restricting yourself to data from 1998 (after treatment), calculate the average enrollment rate among **poor** households in the Treatment villages and the average enrollment rate among **poor** households in the control villages. Use a t-test to determine if this difference is statistically significant. What do you conclude?

In [ ]:
# Your code here

*Discuss your results here*

### 2.2 Simple differences: Regression

Estimate the effects of Progresa on enrollment using a regression model, by regressing the 1998 enrollment rates **of the poor** on treatment assignment. Discuss the following:

* Based on this model, how much did Progresa increase the likelihood of a child enrolling?
* How does your regression estimate compare to your t-test estimate from part 2.1?
* Based on this regression model, can we reject the null hypothesis that the treatment effects are zero? 
* What is the counterfactual assumption underlying this regression?

In [ ]:
# Your code here

*Discuss your results here*

### 2.3 Multiple Regression

Re-run the above regression estimated but this time include a set of control variables. Include, for instance, age, distance to a secondary school, gender, education of household head, welfare index, indigenous, etc.

* How do the controls affect the point estimate of treatment effect?
* How do the controls affect the standard error on the treatment effect? 
* How do you interpret the differences (or similarities) between your estimates of 2.2 and 2.3?

In [ ]:
# Your code here

*Discuss your results here*

### 2.4 Difference-in-Difference, version 1 (tabular)

Thus far, we have computed the effects of Progresa by estimating the difference in 1998 enrollment rates across villages. An alternative approach would be to compute the treatment effect using a difference-in-differences framework.

Begin by estimating the average treatment effects of the program for poor households using data from 1997 and 1998. Specifically, calculate the difference (between 1997 and 1998) in enrollment rates among poor households in treated villages; then compute the difference (between 1997 and 1998) in enrollment rates among poor households in control villages. The difference between these two differences is your estimate.

* What is your estimate of the impact, and how does it compare to your earlier (simple difference) results?
* What is the counterfactual assumption underlying this estimate? 


In [ ]:
# Your code here

*Discuss your results here*

### 2.5 Difference-in-Difference, version 2 (regression)

Now use a regression specification to estimate the average treatment effects of the program in a difference-in-differences framework. Include at least 5 control variables.

* What is your estimate of the impact of Progresa? Be very specific in interpreting your coefficients and standard errors, and make sure to specify exactly what units you are measuring and estimating.
* How do these estimates of the treatment effect compare to the estimates based on the simple difference?
* How do these estimates compare to the difference-in-difference estimates from 2.4 above? What accounts for these differences?
* What is the counterfactual assumption underlying this regression? 

In [ ]:
# Your code here

*Discuss your results here*

### 2.6 Summary

* Based on all the analysis you have undertaken to date, do you believe that Progresa had a causal impact on the enrollment rates of poor households in Mexico? 
* Describe one other way that you might analyze these data to further investigate the causal impact of Progresa on enrollment, and clearly state the counterfactual assumption you would need to make for that approach to be valid.  *(Hint: Consider using the non-poor in your analysis)*

*Discuss your results here*